In [1]:
import re
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Video
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

In [15]:
# ===== Conexiones MediaPipe =====

# Manos (21 keypoints)
HAND_CONNECTIONS = [
    (0,1),(1,2),(2,3),(3,4),          # pulgar
    (0,5),(5,6),(6,7),(7,8),          # índice
    (5,9),(9,10),(10,11),(11,12),     # medio
    (9,13),(13,14),(14,15),(15,16),   # anular
    (13,17),(17,18),(18,19),(19,20),  # meñique
    (0,17)                            # palma
]

# --- lista de conexiones pose (15 keypoints) ---
POSE_CONNECTIONS = [
    (0,1),(1,2),(2,3),        # lado izquierdo ojo-nariz
    (0,4),(4,5),(5,6),        # lado derecho ojo-nariz
    (3,7), (6,8),             # ojos ↔ orejas
    (9,10),            # boca ↔ nariz
    (11,12),                  # hombros
    (11,13), (12,14)          # hombros ↔ codos
]

# ===== Utilidades =====

def _detect_frames(cols):
    """
    Detecta el número de frames (T) a partir de sufijos _F1 .. _F{T}.
    """
    fnums = set()
    for c in cols:
        m = re.search(r"_F(\d+)$", c)
        if m:
            fnums.add(int(m.group(1)))
    if not fnums:
        raise ValueError("No se encontraron sufijos '_F#' en las columnas.")
    return sorted(fnums)  # p.ej. [1..T]

def _collect_part_coords(row, part_prefix, frame_idx):
    """
    Extrae coords (x,y) para una parte (pose_, hand_l_, hand_r_, face_) en un frame dado (1-indexed).
    Devuelve ndarray shape (N, 2) con NaN donde falten pares.
    Si no hay columnas para la parte/frame, devuelve None.
    """
    cols_x = [c for c in row.index if c.startswith(part_prefix) and f"_X_F{frame_idx}" in c]
    cols_y = [c for c in row.index if c.startswith(part_prefix) and f"_Y_F{frame_idx}" in c]
    if not cols_x or not cols_y:
        return None

    # Alinear por índice K# usando el prefijo '..._K{idx}_X_Ft' y '..._K{idx}_Y_Ft'
    def _kid(c):  # extrae K#
        m = re.search(r"_K(\d+)_", c)
        return int(m.group(1)) if m else None
    xs = sorted(cols_x, key=_kid)
    ys = sorted(cols_y, key=_kid)

    # Emparejar por orden K#
    n = min(len(xs), len(ys))
    coords = []
    for i in range(n):
        xv = row[xs[i]]
        yv = row[ys[i]]
        coords.append([xv, yv])
    return np.array(coords, dtype=float) if coords else None

def _plot_part(ax, coords, connections=None, point_color="green", line_color="k",
               point_size=12, line_width=1.5):
    """Dibuja puntos y (opcional) conexiones si hay suficientes índices."""
    if coords is None or coords.size == 0:
        return
    # Puntos (omite NaN)
    mask = ~np.isnan(coords).any(axis=1)
    if mask.any():
        ax.scatter(coords[mask,0], coords[mask,1], c=point_color, s=point_size)
    # Conexiones
    if connections:
        for i, j in connections:
            if i < len(coords) and j < len(coords):
                if not (np.isnan(coords[i]).any() or np.isnan(coords[j]).any()):
                    ax.plot([coords[i,0], coords[j,0]], [coords[i,1], coords[j,1]],
                            color=line_color, linewidth=line_width)

def plot_holistic_video_from_row(
    df_row,                     # una fila (Series) del DataFrame con columnas pose_, hand_l_, hand_r_, face_
    output_path="holistic_plot.mp4",
    fps=8,
    frame_size=(720, 720),
    draw_pose=True,
    draw_left_hand=True,
    draw_right_hand=True,
    colors=dict(pose="deepskyblue", left="limegreen", right="orange", face="crimson"),
    point_sizes=dict(pose=10, left=18, right=18, face=2),
    line_colors=dict(pose="navy", left="black", right="black"),
    line_widths=dict(pose=2.0, left=2.0, right=2.0),
    show_title=True,
    label=""
):
    """
    Genera un video con todos los keypoints disponibles (pose, mano izq/der, face) provenientes de una fila del DF.
    Detecta automáticamente la cantidad de frames por sufijos '_F#'.

    df_row: pd.Series con columnas tipo:
        'pose_K{i}_X_Ft', 'pose_K{i}_Y_Ft',
        'hand_l_K{i}_X_Ft', 'hand_l_K{i}_Y_Ft',
        'hand_r_K{i}_X_Ft', 'hand_r_K{i}_Y_Ft',
        'face_K{i}_X_Ft', 'face_K{i}_Y_Ft'
    """

    cols = df_row.index.tolist()
    frames = _detect_frames(cols)  # p.ej. [1..T]

    # === calcular límites globales (min/max) con TODO lo disponible ===
    all_xy = []
    for t in frames:
        for pref in ("pose_", "hand_l_", "hand_r_", "face_"):
            coords = _collect_part_coords(df_row, pref, t)
            if coords is not None and coords.size > 0:
                # filtra filas NaN
                m = ~np.isnan(coords).any(axis=1)
                if m.any():
                    all_xy.append(coords[m])
    if not all_xy:
        raise ValueError("No se encontraron coordenadas válidas en ninguna parte/frame.")

    all_xy = np.vstack(all_xy)  # (M,2)
    x_min, x_max = np.min(all_xy[:,0]), np.max(all_xy[:,0])
    y_min, y_max = np.min(all_xy[:,1]), np.max(all_xy[:,1])

    # === VideoWriter H.264 ===
    fourcc = cv2.VideoWriter_fourcc(*'avc1')  # H.264 (si tu OpenCV+FFmpeg lo soporta)
    out = cv2.VideoWriter(output_path, fourcc, fps, frame_size)

    # === Generar cada frame del video ===
    for idx, t in enumerate(frames):
        fig, ax = plt.subplots(figsize=(7,7))
        canvas = FigureCanvas(fig)

        # Pose
        if draw_pose:
            pose_coords = _collect_part_coords(df_row, "pose_", t)
            _plot_part(ax, pose_coords, POSE_CONNECTIONS,
                       point_color=colors.get("pose","deepskyblue"),
                       line_color=line_colors.get("pose","navy"),
                       point_size=point_sizes.get("pose",10),
                       line_width=line_widths.get("pose",2.0))

        # Mano izquierda
        if draw_left_hand:
            lh_coords = _collect_part_coords(df_row, "hand_l_", t)
            _plot_part(ax, lh_coords, HAND_CONNECTIONS,
                       point_color=colors.get("left","limegreen"),
                       line_color=line_colors.get("left","black"),
                       point_size=point_sizes.get("left",18),
                       line_width=line_widths.get("left",2.0))

        # Mano derecha
        if draw_right_hand:
            rh_coords = _collect_part_coords(df_row, "hand_r_", t)
            _plot_part(ax, rh_coords, HAND_CONNECTIONS,
                       point_color=colors.get("right","orange"),
                       line_color=line_colors.get("right","black"),
                       point_size=point_sizes.get("right",18),
                       line_width=line_widths.get("right",2.0))
            
        # 🔸 Conexiones extra pose ↔ manos
        if pose_coords is not None:
            if draw_left_hand and lh_coords is not None:
                if len(pose_coords) > 14 and len(lh_coords) > 0:
                    if not (np.isnan(pose_coords[14]).any() or np.isnan(lh_coords[0]).any()):
                        ax.plot([pose_coords[14,0], lh_coords[0,0]],
                                [pose_coords[14,1], lh_coords[0,1]],
                                color="purple", linewidth=2.0, linestyle="--")

            if draw_right_hand and rh_coords is not None:
                if len(pose_coords) > 13 and len(rh_coords) > 0:
                    if not (np.isnan(pose_coords[13]).any() or np.isnan(rh_coords[0]).any()):
                        ax.plot([pose_coords[13,0], rh_coords[0,0]],
                                [pose_coords[13,1], rh_coords[0,1]],
                                color="purple", linewidth=2.0, linestyle="--")

        # Ajustes de ejes
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
        ax.invert_yaxis()
        ax.set_xticks([]); ax.set_yticks([])
        ax.set_aspect('equal', adjustable='box')

        if show_title:
            ax.set_title(f"{label}  |  Frame {idx}/{len(frames) - 1}")

        # Renderizar y volcar a OpenCV
        canvas.draw()
        img = np.asarray(canvas.buffer_rgba())
        img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
        img = cv2.resize(img, frame_size)
        if idx != 0:
            out.write(img)
        plt.close(fig)

    out.release()
    print(f"🎞️ Video guardado en: {output_path}")


In [3]:
df = pd.read_csv("./Dataset.csv")

In [19]:
# Seleccionar una fila aleatoria de todo el DataFrame
row = df.sample(1).iloc[0]

# Generar el video
plot_holistic_video_from_row(
    df_row=row,
    output_path="holistic.mp4",
    fps=8,
    frame_size=(720,720),
    draw_pose=True,
    draw_left_hand=True,
    draw_right_hand=True,
    label=f"Clase {row['class']}" if "class" in row else "Sample"
)

# Mostrar el video en el notebook
Video("holistic.mp4", embed=True, width=600, height=600)

🎞️ Video guardado en: holistic.mp4
